In [1]:
import pandas as pd
import numpy as np
import h5py
import math

from tokenizers import ByteLevelBPETokenizer

In [2]:
samples = pd.read_csv('Reviews.csv')

In [3]:
samples['x'] = samples['Review'].str.lower()
samples['x'] = samples['x'].str.replace(r'[^a-z\s\']', ' ', regex=True)
samples['x'] = samples['x'].str.replace(r'\'', '', regex=True)
samples['x'] = samples['x'].str.replace(r'\s+', ' ', regex=True)
samples['x']

0        working with one of the best shakespeare sourc...
1        well tremors i the original started off in and...
2        ouch this one was a bit painful to sit through...
3        ive seen some crappy movies in my life but thi...
4         carriers follows the exploits of two guys and...
                               ...                        
49995    this movie is certainly well constructed begin...
49996    nice to see a comedy for grown ups masterfully...
49997    jean renoirs homage to the paris of the late t...
49998    what are the movies i mean what are movies mad...
49999    i saw this movie on tv and loved it i am a rea...
Name: x, Length: 50000, dtype: object

In [ ]:
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(samples['x'], vocab_size=4096, min_frequency=64)

In [5]:
len(tokenizer.get_vocab())

4096

In [6]:
samples['x'] = samples['x'].apply(lambda x: tokenizer.encode(x).ids)
tokenized_length = samples['x'].apply(len).max()
samples['x'] = samples['x'].apply(lambda x: (([0]*tokenized_length)+x)[-tokenized_length:])
samples['x'] = samples['x'].apply(lambda x: np.array(x))

tokenized_length

3675

In [7]:
x = np.array(samples['x'].tolist()).astype(np.uint16)
y = np.array(samples['Sentiment'].tolist()).astype(np.uint8)

In [9]:
file = h5py.File('xy.h5', 'w')

file.create_dataset('x', data=x)
file.create_dataset('y', data=y)

file.close()